In [36]:
import streamlit as st
from langchain_core.messages.chat import ChatMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import load_prompt
from langchain.memory import ConversationBufferMemory
from langchain import hub

from dotenv import load_dotenv

from langchain_teddynote import logging
import time
import base64
# from streamlit_pdf_viewer import pdf_viewer
from urllib.parse import urlencode

from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# _type: "prompt"
# template: |
#   You are an assistant for question-answering tasks.
#   Use the following pieces of retrieved `information` to answer the question with emoji. 
#   If you don't know the answer, just say that you don't know. 
  

#   <information>
#   {context}
#   </information>

#   #Question:
#   {question} 

#   #Answer:
# input_variables: ["question", "context"]

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

model_name_path = '../../HUGGING_FACE_MODEL/BAAI_bge-m3'

def content_for_documents(document_list):
    return "\n\n".join([doc.page_content for doc in document_list])

embeddings = HuggingFaceEmbeddings(
        model_name=model_name_path, 
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs={
            "normalize_embeddings": True
        },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
        # cache_folder='../embedding/model',
    )

# Chroma db Loading
chroma_db = Chroma(persist_directory="../../Chroma_DB/chroma_bank_law_db",
                embedding_function=embeddings, 
                collection_name="bank_law_case")

# Retriever 정의
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

prompt_type = ""
# prompt_type = "은행"

if prompt_type == "은행":
    # prompt = load_prompt("../prompts/rag.yaml")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions."
                # "A chat between a curious user and artificial intelligence assistant. The assistant gives helpfule, detailed, and polite answer to the user's questions."                    
                # "당신은 친절한 20년차 은행원이면서 IT 전문가인 JABIS 입니다. 다음의 질문에 답변해 주세요.",
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            (
                "user", 
                """Human: <information>{context}</information>\n\n
                   #Question: {question}\nAssistant: """
            ),

        ]
    )

    # Embedding Call
else:
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions."
                # "A chat between a curious user and artificial intelligence assistant. The assistant gives helpfule, detailed, and polite answer to the user's questions."                    
                # "당신은 친절한 20년차 은행원이면서 IT 전문가인 JABIS 입니다. 다음의 질문에 답변해 주세요.",
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "Human: {question}\nAssistant: "),
        ]
    )


runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

if prompt_type == "은행":
    chain = (
        {"context": retriever | content_for_documents, "question": RunnablePassthrough(), "chat_history":runnable}
        | prompt
        | llm
        | output_parser
    )
else:
    # from langchain_core.runnables import RunnableLambda
    # from operator import itemgetter
    chain = (
        runnable
        | prompt
        | llm
        | output_parser
    )

# user_input = "대한민국의 수도는?"
# ai_answer = chain.invoke({"question": input})

# memory.save_context({"human": user_input}, {"assistant": ai_answer})

In [9]:
runnable.invoke({"input": "hi!"})

{'input': 'hi!', 'chat_history': []}

In [17]:
chain = (
    {"context": retriever | content_for_documents, "question": RunnablePassthrough(), "chat_history":runnable}
    | prompt | llm | output_parser)

# chain = (runnable | prompt | llm | output_parser)


In [12]:
user_input = "서울의 수도가 어디야?"
user_input = "내 이름은 이현우야 만나서 반가워"
ai_answer = chain.invoke({"question": user_input})

memory.save_context({"human": user_input}, {"assistant": ai_answer})


In [15]:
user_input = "내 이름이 뭐라고?"
chain.invoke({"question": user_input})


'당신의 이름은 이현우입니다. 만나서 반가워요, 이현우님! 궁금한 점이나 도움이 필요하시면 언제든 말씀해주세요. 최선을 다해 도와드리겠습니다.'

In [16]:
chain

RunnableAssign(mapper={
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
})
| ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions.")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Human: {question}\nAssistant: '))])
| ChatOllama(model='EEVE-Korean-10.8B:latest')
| StrOutputParser()

In [18]:
chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x17eaf2190>, search_kwargs={'k': 2})
           | RunnableLambda(content_for_documents),
  question: RunnablePassthrough(),
  chat_history: RunnableAssign(mapper={
                  chat_history: RunnableLambda(load_memory_variables)
                                | RunnableLambda(itemgetter('chat_history'))
                })
}
| ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="A chat between a curious user and artificial intelligen

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions."
            # "A chat between a curious user and artificial intelligence assistant. The assistant gives helpfule, detailed, and polite answer to the user's questions."                    
            # "당신은 친절한 20년차 은행원이면서 IT 전문가인 JABIS 입니다. 다음의 질문에 답변해 주세요.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "user", 
            """Human: <information>{context}</information>\n\n
                #Question: {question}\nAssistant: """
        ),

    ]
)

chain = (
    {"context": retriever | content_for_documents, "question": RunnablePassthrough()}
    | prompt | llm | output_parser)


In [34]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)    
runnable = RunnableParallel(
    context=retriever | content_for_documents,
    question=RunnablePassthrough(),
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.

)


In [28]:
runnable

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x17eaf2190>, search_kwargs={'k': 2})
           | RunnableLambda(content_for_documents),
  question: RunnableLambda(RunnablePassthrough),
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
}

In [ ]:
runnable.invoke({"question": "서울의 수도가 어디니?"})

In [22]:
chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x17eaf2190>, search_kwargs={'k': 2})
           | RunnableLambda(content_for_documents),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions.")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['c

In [52]:
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [61]:
model_name_path = '../../HUGGING_FACE_MODEL/BAAI_bge-m3'

def content_for_documents(document_list):
    return "\n\n".join([doc.page_content for doc in document_list])

embeddings = HuggingFaceEmbeddings(
        model_name=model_name_path, 
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs={
            "normalize_embeddings": True
        },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
        # cache_folder='../embedding/model',
    )

# Chroma db Loading
chroma_db = Chroma(persist_directory="../../Chroma_DB/chroma_bank_law_db",
                embedding_function=embeddings, 
                collection_name="bank_law_case")

# Retriever 정의
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

# runnable = RunnableParallel(
#     chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions."
            # "A chat between a curious user and artificial intelligence assistant. The assistant gives helpfule, detailed, and polite answer to the user's questions."                    
            # "당신은 친절한 20년차 은행원이면서 IT 전문가인 JABIS 입니다. 다음의 질문에 답변해 주세요.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "Human: {question}\nAssistant: "),
    ]
)

chain = (
    runnable
    | prompt
)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [60]:
chain
# chain.invoke({"question":"서울의 수도가 어디니?"})

RunnableAssign(mapper={
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
})
| ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions.")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Human: {question}\nAssistant: '))])

In [57]:
chain
# chain.invoke({"question":"서울의 수도가 어디니?"})

{
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
}
| ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="A chat between a curious user and artificial intelligence assistant. The assistant gives simple answer to the user's questions.")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Human: {question}\nAssistant: '))])

In [ ]:
user_input = "대한민국의 수도는 어디야?"
ai_answer = "대한민국의 수도는 전주야"

memory.save_context({"human": user_input}, {"assistant": ai_answer})

In [38]:
runnable

RunnableAssign(mapper={
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
})

In [39]:
# runnable = RunnableParallel(
#     context=retriever | content_for_documents,
#     question=RunnablePassthrough(),
#     chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
# )

runnable = RunnableParallel(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)


In [40]:
runnable

{
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
}